In [63]:
import requests, json
from bs4 import BeautifulSoup as BS
import pandas as pd
import geocoder
import numpy as np
import re
from country_codes import country_codes, alt_country

In [64]:
def country_name_lookup(df, dictionary):
    # Create a reverse mapping from abbreviation to country name
    abbr_to_country = {abbr: country for country, abbrs in dictionary.items() for abbr in abbrs}
    # Map the abbreviations in the DataFrame to their full country names
    df['Country'] = df['Country'].map(abbr_to_country).fillna(df['Country'])
    return df

def getTable(url, table_num=0):
    url_content = requests.get(url).content
    url_data = re.sub(r'<.*?>', lambda g: g.group(0).upper(), str(url_content))
    url_html = BS(url_data, 'html.parser')
    html_table = url_html.find_all('table')[0]
    meta_df = pd.read_html(str(html_table))
    records = meta_df[0].to_json(orient='records')
    data = json.loads(records)
    df= pd.read_json(json.dumps(data))
    return df


In [65]:
locations = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
country_loc_df = getTable(locations)
country_loc_df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [66]:
covid_data_url = 'https://www.worldometers.info/coronavirus/#countries'
covid_country_table= getTable(covid_data_url)
covid_country_table

,#,"Country, Other",Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,"Serious, Critical",...,Total Tests,Tests/ \n 1M pop\n,Population,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,NaN,\n North America\n,131889132,NaN,1695941.0,NaN,127665129.0,350.0,2528062.0,6095.0,...,NaN,NaN,NaN,North America,\n,NaN,NaN,NaN,NaN,NaN
1,NaN,\n Asia\n,221500265,NaN,1553662.0,NaN,205673091.0,NaN,14273512.0,14733.0,...,NaN,NaN,NaN,Asia,\n,NaN,NaN,NaN,NaN,NaN
2,NaN,\n Europe\n,253406198,NaN,2101824.0,NaN,248754104.0,474.0,2550270.0,4453.0,...,NaN,NaN,NaN,Europe,\n,NaN,NaN,NaN,NaN,NaN
3,NaN,\n South America\n,70200879,NaN,1367332.0,NaN,66683585.0,NaN,2149962.0,8953.0,...,NaN,NaN,NaN,South America,\n,NaN,NaN,NaN,NaN,NaN
4,NaN,\n Oceania\n,14895771,NaN,33015.0,NaN,14752388.0,NaN,110368.0,31.0,...,NaN,NaN,NaN,Australia/Oceania,\n,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,Total:,70200879,NaN,1367332.0,NaN,66683585.0,NaN,2149962.0,8953.0,...,NaN,NaN,NaN,South America,None,NaN,NaN,NaN,NaN,NaN
243,NaN,Total:,14895771,NaN,33015.0,NaN,14752388.0,NaN,110368.0,31.0,...,NaN,NaN,NaN,Australia/Oceania,None,NaN,NaN,NaN,NaN,NaN
244,NaN,Total:,12860924,NaN,258892.0,NaN,12090808.0,NaN,511224.0,529.0,...,NaN,NaN,NaN,Africa,None,NaN,NaN,NaN,NaN,NaN
245,NaN,Total:,721,NaN,15.0,NaN,706.0,NaN,0.0,0.0,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN


In [67]:
covid_country_table.drop([0,1,2,3,4,5,6,7], inplace=True)

covid_df = covid_country_table.sort_values(by='Total Cases', ascending = False).reset_index(drop=True)
covid_df.dropna(subset=['#'],inplace= True)


covid_df.rename(columns={'Country, Other':'Country',
                          'Total Recovered': 'Recoveries',
                          'Total Cases':'Cases',
                          'Total Deaths':'Deaths',
                           '#':'Rank',}, inplace = True)

country_name_lookup(covid_df, alt_country)
covid_df.reset_index(drop=True, inplace=True)

covid_df

,Rank,Country,Cases,New Cases,Deaths,New Deaths,Recoveries,New Recovered,Active Cases,"Serious, Critical",...,Total Tests,Tests/ \n 1M pop\n,Population,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,1.0,United States,111820082,NaN,1219487.0,NaN,109814428.0,NaN,786167.0,940.0,...,1.186852e+09,3544901.0,3.348053e+08,North America,3,275.0,0.0,NaN,NaN,2348.0
1,2.0,India,45035393,NaN,533570.0,NaN,NaN,NaN,NaN,NaN,...,9.358795e+08,665334.0,1.406632e+09,Asia,31,2636.0,2.0,NaN,NaN,0.4
2,3.0,France,40138560,NaN,167642.0,NaN,39970918.0,NaN,0.0,NaN,...,2.714902e+08,4139547.0,6.558452e+07,Europe,2,391.0,0.0,NaN,NaN,NaN
3,4.0,Germany,38828995,NaN,183027.0,NaN,38240600.0,NaN,405368.0,NaN,...,1.223324e+08,1458359.0,8.388360e+07,Europe,2,458.0,1.0,NaN,NaN,4833.0
4,5.0,Brazil,38743918,NaN,711380.0,NaN,36249161.0,NaN,1783377.0,NaN,...,6.377617e+07,296146.0,2.153536e+08,South America,6,303.0,3.0,NaN,NaN,8281.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,226.0,Diamond Princess,712,NaN,13.0,NaN,699.0,NaN,0.0,NaN,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN
227,227.0,Tokelau,80,NaN,NaN,NaN,NaN,NaN,80.0,NaN,...,NaN,NaN,1.378000e+03,Australia/Oceania,17,NaN,NaN,NaN,NaN,58055.0
228,228.0,Vatican City,29,NaN,NaN,NaN,29.0,NaN,0.0,NaN,...,NaN,NaN,7.990000e+02,Europe,28,NaN,NaN,NaN,NaN,NaN
229,229.0,Western Sahara,10,NaN,1.0,NaN,9.0,NaN,0.0,NaN,...,NaN,NaN,6.261610e+05,Africa,62616,626161.0,NaN,NaN,NaN,NaN


In [68]:
country_code_df = pd.DataFrame(country_codes)
country_code_df.rename(columns={'name':'Country'}, inplace=True)
country_loc_df.rename(columns={'name':'Country'}, inplace=True)


full_df = pd.merge(recover_df, country_loc_df, on='Country')
full_df['Rank'] = full_df['Rank'].astype(int)
code_df = pd.merge(full_df, country_code_df, on='Country')


flag_urls = [f'https://cdn.jsdelivr.net/npm/country-flag-icons@1.5.7/1x1/{code}.svg' for code in code_df['code']]
flag_urls = pd.DataFrame(flag_urls)
flag_urls.columns = ['flag_url']
flag_urls['Country'] =code_df['Country']
full_df = pd.merge(flag_urls, full_df, on ='Country')
full_df = full_df.replace(np.nan, 0)

In [69]:
full_df

,flag_url,Country,Rank,Cases,New Cases,Deaths,New Deaths,Recoveries,New Recovered,Active Cases,...,Continent,1 Case every X ppl,1 Death every X ppl,1 Test every X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop,country,latitude,longitude
0,https://cdn.jsdelivr.net/npm/country-flag-icon...,United States,1,111820082,0.0,1219487.0,0.0,109814428.0,0.0,786167.0,...,North America,3,275.0,0.0,0.0,0.0,2348.0,US,37.090240,-95.712891
1,https://cdn.jsdelivr.net/npm/country-flag-icon...,India,2,45035393,0.0,533570.0,0.0,0.0,0.0,0.0,...,Asia,31,2636.0,2.0,0.0,0.0,0.4,IN,20.593684,78.962880
2,https://cdn.jsdelivr.net/npm/country-flag-icon...,France,3,40138560,0.0,167642.0,0.0,39970918.0,0.0,0.0,...,Europe,2,391.0,0.0,0.0,0.0,0.0,FR,46.227638,2.213749
3,https://cdn.jsdelivr.net/npm/country-flag-icon...,Germany,4,38828995,0.0,183027.0,0.0,38240600.0,0.0,405368.0,...,Europe,2,458.0,1.0,0.0,0.0,4833.0,DE,51.165691,10.451526
4,https://cdn.jsdelivr.net/npm/country-flag-icon...,Brazil,5,38743918,0.0,711380.0,0.0,36249161.0,0.0,1783377.0,...,South America,6,303.0,3.0,0.0,0.0,8281.0,BR,-14.235004,-51.925280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,https://cdn.jsdelivr.net/npm/country-flag-icon...,Saint Helena,222,2166,0.0,0.0,0.0,2.0,0.0,2164.0,...,Africa,3,0.0,0.0,0.0,0.0,353884.0,SH,-24.143474,-10.030696
191,https://cdn.jsdelivr.net/npm/country-flag-icon...,Montserrat,224,1403,0.0,8.0,0.0,1376.0,0.0,19.0,...,North America,4,621.0,0.0,0.0,0.0,3827.0,MS,16.742498,-62.187366
192,https://cdn.jsdelivr.net/npm/country-flag-icon...,Niue,225,1059,0.0,0.0,0.0,1056.0,0.0,3.0,...,Australia/Oceania,2,0.0,0.0,0.0,0.0,1850.0,NU,-19.054445,-169.867233
193,https://cdn.jsdelivr.net/npm/country-flag-icon...,Tokelau,227,80,0.0,0.0,0.0,0.0,0.0,80.0,...,Australia/Oceania,17,0.0,0.0,0.0,0.0,58055.0,TK,-8.967363,-171.855881
